In [1]:
import pandas as pd
import numpy as np
import os
import trimesh
from meshparty import trimesh_io, trimesh_vtk, skeleton, utils
import vtk
import matplotlib.pyplot as plt
import datetime

In [2]:
# Load astrocyte cellids into a list
astro_list = [864691136194086870, 864691134964446239, 864691135012524790, 864691135256138671, 864691135348239831, 864691135809446092, 864691135880405261, 864691136023933241]
neuro_list = [864691135269913253, 864691135337851366, 864691135341421745, 864691135386363265, 864691135497750291, 864691135657783170, 864691135837182867, 864691136194008022]
vasc_list = [864691136330394007]

print(f'number of astros:', len(astro_list))
print(f'number of neurons:', len(neuro_list))
print(f'number of vascular:', len(vasc_list))

number of astros: 8
number of neurons: 8
number of vascular: 1


In [3]:
mesh_dir = 'data/cell_meshes/dec/' # or change to your desired folder
seg_source = "precomputed://gs://iarpa_microns/minnie/minnie65/seg"
mm = trimesh_io.MeshMeta(cv_path=seg_source,
                         disk_cache_path=mesh_dir, 
                         cache_size=20)

In [4]:
# Combine all the lists into a dictionary with their respective prefixes
prefix_dict = {
    'astro_': astro_list,
    'neuro_': neuro_list,
    'vasc_': vasc_list
}

# Make a dictionary to hold the mesh file for each cell id in the lists
mesh_dictionary = {}
for prefix, cell_list in prefix_dict.items():
    for cellid in cell_list:
        mesh_file = os.path.join(mesh_dir, str(cellid) + '_0.ply')
        mesh_dictionary[prefix + str(cellid)] = trimesh.load_mesh(mesh_file)


In [5]:
len(mesh_dictionary)

17

In [6]:
# Opacity settings for each cell type
astro_opac = 0.15
neuro_opac = 0.5
vasc_opac = 0.05

# Define the opacity for each prefix
opacity_dict = {
    'astro_': astro_opac,
    'neuro_': neuro_opac,
    'vasc_': vasc_opac
}

# Creating mesh actors with opacity and random colors
mesh_actor = {}
centroids = []
for prefix, cell_list in prefix_dict.items():
    for cellid in cell_list:
        cell_key = prefix + str(cellid)
        if cell_key in mesh_dictionary:
            random_color = list(np.random.random(size=3))
            cell_opac = opacity_dict[prefix]
            mesh_actor[cell_key] = trimesh_vtk.mesh_actor(mesh_dictionary[cell_key], opacity=cell_opac, color=random_color)
            centroids.append(mesh_dictionary[cell_key].centroid)

locals().update(mesh_actor)

# Calculate global mean centroid
if centroids:
    global_mean_centroid = np.mean(centroids, axis=0)
else:
    global_mean_centroid = np.array([0, 0, 0])
    print("No centroids available for mean calculation.")

# Creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(global_mean_centroid, backoff=900)

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x000002593B229F90) at 0x00000259402882E8>

In [ ]:
# save image to disk

save_dir = 'vtk_images/'
base_name = 'minnie_astro_neuro_vasc_'
extension = 'png'

date_time_stamp = datetime.datetime.now().strftime("%Y_%m_%d_%H%M_%S")
filename = f"{save_dir}{base_name}_{date_time_stamp}.{extension}"

# uncomment to save

# trimesh_vtk.render_actors(mesh_actor.values(),
#                           filename=filename,
#                           do_save=True,
#                           scale=6,
#                           camera=camera
#                          )